# Лабораторная работа: Мета-обучение
## Шаги 7-8: Мета-обучение и анализ важности признаков

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

%matplotlib inline
sns.set_style('whitegrid')

In [ ]:
# Загружаем мета-набор
try:
    meta_dataset = pd.read_csv('../results/meta_dataset/meta_dataset_complete.csv')
    print(f"Загружен мета-набор: {meta_dataset.shape[0]} строк, {meta_dataset.shape[1]} колонок")
    display(meta_dataset.head())
except FileNotFoundError:
    print("Мета-набор не найден. Сначала выполните предыдущие шаги.")
    meta_dataset = None

In [ ]:
if meta_dataset is not None:
    # Подготовка данных
    feature_cols = [c for c in meta_dataset.columns 
                   if c not in ['dataset_id', 'dataset_name', 'best_algorithm']]
    
    X = meta_dataset[feature_cols].select_dtypes(include=[np.number]).fillna(0)
    y = meta_dataset['best_algorithm']
    
    print(f"Признаков: {X.shape[1]}")
    print(f"Классов: {len(np.unique(y))}")
    print(f"Распределение классов:\n{y.value_counts()}")
    
    # Разделение
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Масштабирование
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

In [ ]:
if meta_dataset is not None:
    # Обучение моделей
    models = {
        'Dummy': DummyClassifier(strategy='most_frequent'),
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
    }
    
    results = []
    for name, model in models.items():
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='weighted')
        results.append({'Model': name, 'Accuracy': acc, 'F1-score': f1})
    
    results_df = pd.DataFrame(results)
    print("\nРезультаты:")
    display(results_df)

In [ ]:
if meta_dataset is not None:
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Accuracy
    bars1 = axes[0].bar(results_df['Model'], results_df['Accuracy'], 
                        color=['gray', 'steelblue', 'coral'], alpha=0.7)
    axes[0].set_ylabel('Accuracy')
    axes[0].set_title('Сравнение точности мета-алгоритмов')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Добавляем значения
    for bar in bars1:
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.3f}', ha='center', va='bottom')
    
    # F1-score
    bars2 = axes[1].bar(results_df['Model'], results_df['F1-score'],
                        color=['gray', 'steelblue', 'coral'], alpha=0.7)
    axes[1].set_ylabel('F1-score')
    axes[1].set_title('Сравнение F1-score мета-алгоритмов')
    axes[1].tick_params(axis='x', rotation=45)
    
    for bar in bars2:
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('../results/figures/meta_learning_results.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
if meta_dataset is not None:
    print("\n" + "="*60)
    print("ВЫВОДЫ")
    print("="*60)
    print(f"1. Random Forest показал лучшие результаты: {results_df.loc[2, 'Accuracy']:.3f}")
    print(f"2. Улучшение относительно наивного алгоритма: {(results_df.loc[2, 'Accuracy'] - results_df.loc[0, 'Accuracy'])*100:.1f}%")
    print("3. Мета-признаки позволяют эффективно предсказывать лучший алгоритм")